In [2]:
from tensorflow import keras
import matplotlib.pyplot as plt
import pymysql as sql
import pandas as pd
import numpy as np

In [3]:
sql1 ='''select * from parameter'''
sql2='''select *from Sensor_Data order by date desc limit 4'''

In [5]:
db1=sql.connect(host='cjw9507.iptime.org',
                user='user1',password='pw',
                db='MSC',charset='utf8')
db2=sql.connect(host='cjw9507.iptime.org',
                user='user1',password='pw',
                db='MSC',charset='utf8')
db3=sql.connect(host='cjw9507.iptime.org',
                user='user1',password='pw',
                db='MSC',charset='utf8')

In [6]:
curs1=db1.cursor()
curs2=db2.cursor()
curs3=db3.cursor()

In [7]:
curs1.execute(sql1)

1

In [11]:
data=curs1.fetchall()

In [35]:
MinTemp=data[0][0]
MinHumi=data[0][1]
MaxTemp=data[0][2]
MaxHumi=data[0][3]

In [39]:
curs2.execute(sql2)
data2=curs2.fetchall()

In [41]:
new_data2=[]
for i in data2:
    new_data2.append(i)

In [43]:
new_data2.reverse()

In [45]:
processing_data2=[]
for i in new_data2:
    sub=[]
    sub.append(i[1])
    sub.append(i[2])
    processing_data2.append(sub)

In [47]:
processing_data2=np.array(processing_data2)

In [50]:
def MinMaxReturn(val, Min, Max):
    ''' Return normalized data '''
    return val * (Max - Min + 1e-7) + Min
def FindMinMax(data):
    ''' Find Minimum & Maximum value '''
    Mindata = np.min(data, 0)
    Maxdata = np.max(data, 0)
    return Mindata, Maxdata
def MinMaxScaler(data):
#     global minValue
#     global maxValue
    minValue,maxValue=FindMinMax(data)
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [51]:
processing_data2=MinMaxScaler(processing_data2)

In [61]:
processing_data2=processing_data2.reshape(1,4,2)

In [62]:
processing_data2.shape

(1, 4, 2)

In [54]:
load_model=keras.models.load_model('lstm_epoch300.h5')

In [64]:
predict=load_model.predict(processing_data2)

In [65]:
predict=MinMaxReturn(predict,MinTemp,MaxTemp)

In [81]:
predict[0][0]

35.997704

In [109]:
import datetime
time=str(data2[0][0]+datetime.timedelta(hours=1))
sql3='''INSERT INTO Predict_Data (data,Temp) VALUES ('{}','{}');'''.format(time,predict[0][0])

In [110]:
sql3

"INSERT INTO Predict_Data (data,Temp) VALUES ('2020-06-24 11:00:06','35.997703552246094');"

In [111]:
curs3.execute(sql3)

1

In [113]:
db3.commit()
db3.close()